In [1]:
import numpy as np
from scipy import io

In [2]:
PATH = '../compu/data/'
filename = 'dataset_1.mat'
mat_file = io.loadmat(PATH + filename)
filename = 'angle.mat'
angle_file = io.loadmat(PATH + filename)

In [3]:
S_left = mat_file['S_left']
S_right = mat_file['S_right']
# S_left_phase = mat_file['S_left_phase']
# S_right_phase = mat_file['S_right_phase']

phi = angle_file['phi']
phi = phi[:1000]

In [4]:
# np.savez_compressed('dataset', 
#                     S_left=S_left,
#                     S_right=S_right, 
#                     S_left_phase=S_left_phase, 
#                     S_right_phase=S_right_phase, 
#                     phi=phi)

In [5]:
S_left        = np.transpose(S_left,        (2, 0, 1))
# S_left_phase  = np.transpose(S_left_phase,  (2, 0, 1))
S_right       = np.transpose(S_right,       (2, 0, 1))
# S_right_phase = np.transpose(S_right_phase, (2, 0, 1))

In [6]:
S_left        = np.reshape(S_left,        (1000, 1, 257, 382))
# S_left_phase  = np.reshape(S_left_phase,  (1000, 1, 257, 382))
S_right       = np.reshape(S_right,       (1000, 1, 257, 382))
# S_right_phase = np.reshape(S_right_phase, (1000, 1, 257, 382))

In [7]:
# Log Scale
S_left  = np.log10(S_left  + 1)
S_right = np.log10(S_right + 1)

In [8]:
magnitude = np.concatenate((S_left, S_right), axis=1)
# phase = np.concatenate((S_left_phase, S_right_phase), axis=1)

In [9]:
for i in range(len(magnitude)):
    # Z-score Normalization
    magnitude[i] = np.divide(np.subtract(magnitude[i], magnitude[i].mean()),
                             magnitude[i].std())
#     phase[i]     = np.divide(np.subtract(phase[i]    , phase[i].mean()    ),
#                              phase[i].std())
    
    # Min-Max Normalization
    magnitude[i] = np.divide(np.subtract(magnitude[i], magnitude[i].min()),
                             np.subtract(magnitude[i].max(), magnitude[i].min() ))
#     phase[i]     = np.divide(np.subtract(phase[i]    , phase[i].min()    ),
#                              np.subtract(phase[i].max()    , phase[i].min()     ))

In [10]:
print(magnitude[:,0].shape)
print(magnitude[:,1].shape)

(1000, 257, 382)
(1000, 257, 382)


In [11]:
S_left        = magnitude[:,0].reshape(1000, 1, 257, 382)
S_right       = magnitude[:,1].reshape(1000, 1, 257, 382)
# S_left_phase  = phase[:,0].reshape(1000, 1, 257, 382)
# S_right_phase = phase[:,1].reshape(1000, 1, 257, 382)

In [12]:
inputs = np.concatenate((S_left, S_right), axis=1)

In [13]:
labels = phi.reshape(1000,)

labels[labels == -1] = 200
labels = np.divide(labels, 20)
labels = np.int32(labels)

In [14]:
%whos

Variable     Type       Data/Info
---------------------------------
PATH         str        ../compu/data/
S_left       ndarray    1000x1x257x382: 98174000 elems, type `float64`, 785392000 bytes (749.0081787109375 Mb)
S_right      ndarray    1000x1x257x382: 98174000 elems, type `float64`, 785392000 bytes (749.0081787109375 Mb)
angle_file   dict       n=4
filename     str        angle.mat
i            int        999
inputs       ndarray    1000x2x257x382: 196348000 elems, type `float64`, 1570784000 bytes (1498.016357421875 Mb)
io           module     <module 'scipy.io' from '<...>es/scipy/io/__init__.py'>
labels       ndarray    1000: 1000 elems, type `int32`, 4000 bytes
magnitude    ndarray    1000x2x257x382: 196348000 elems, type `float64`, 1570784000 bytes (1498.016357421875 Mb)
mat_file     dict       n=14
np           module     <module 'numpy' from '/co<...>kages/numpy/__init__.py'>
phi          ndarray    1000x1: 1000 elems, type `int16`, 2000 bytes


In [15]:
print(inputs.shape)
print(inputs.dtype)
print(labels.shape)
print(labels.dtype)

(1000, 2, 257, 382)
float64
(1000,)
int32


In [16]:
# np.savez_compressed('normalized_dataset', inputs=inputs, labels=labels)

In [17]:
def train_test_split(inputs,
                     labels,
                     test_size=0.2,
                     shuffle=True,
                     random_state=None):
    
    test_num = int(inputs.shape[0] * test_size)
    train_num = inputs.shape[0] - test_num

    if shuffle:
        np.random.seed(random_state)
        shuffle = np.random.permutation(inputs.shape[0])
        inputs = inputs[shuffle, :]
        labels = labels[shuffle, ]
        
        inputs_train = inputs[:train_num]
        labels_train = labels[:train_num]
        inputs_test  = inputs[train_num:]
        labels_test  = labels[train_num:]
    else:
        inputs_train = inputs[:train_num]
        labels_train = labels[:train_num]
        inputs_test  = inputs[train_num:]
        labels_test  = labels[train_num:]

    return inputs_train, inputs_test, labels_train, labels_test

In [18]:
inputs_train, inputs_test, labels_train, labels_test  = train_test_split(inputs, labels, test_size=0.2, shuffle=True)
_,            inputs_valid, _,           labels_valid = train_test_split(inputs_train, labels_train, test_size=0.25, shuffle=True)

In [19]:
np.savez_compressed('without_phase_splitted_dataset',
                    inputs_train = inputs_train,
                    inputs_test  = inputs_test,
                    inputs_valid = inputs_valid,
                    labels_train  = labels_train,
                    labels_test   = labels_test,
                    labels_valid  = labels_valid
                   )

In [20]:
%whos

Variable           Type        Data/Info
----------------------------------------
PATH               str         ../compu/data/
S_left             ndarray     1000x1x257x382: 98174000 elems, type `float64`, 785392000 bytes (749.0081787109375 Mb)
S_right            ndarray     1000x1x257x382: 98174000 elems, type `float64`, 785392000 bytes (749.0081787109375 Mb)
angle_file         dict        n=4
filename           str         angle.mat
i                  int         999
inputs             ndarray     1000x2x257x382: 196348000 elems, type `float64`, 1570784000 bytes (1498.016357421875 Mb)
inputs_test        ndarray     200x2x257x382: 39269600 elems, type `float64`, 314156800 bytes (299.603271484375 Mb)
inputs_train       ndarray     800x2x257x382: 157078400 elems, type `float64`, 1256627200 bytes (1198.4130859375 Mb)
inputs_valid       ndarray     200x2x257x382: 39269600 elems, type `float64`, 314156800 bytes (299.603271484375 Mb)
io                 module      <module 'scipy.io' from '